# Modelo ML para la predicción de un plan de teléfono

## 1. Inicialización

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from collections import Counter

## 2. Importar datos

In [2]:
# Cargar el dataset
df = pd.read_csv('users_behavior.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## 3. Separar datos

In [3]:
# Separar los datos en features y target
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [4]:
# Separar los datos para entrenamiento, prueba y validación (60% entrenamiento, 20% validación, 20% prueba)
# Primero, dividimos el dataset en entrenamiento y validación + prueba
features_train, features_valtest, target_train, target_valtest = train_test_split(
    features, target, test_size=0.40, random_state=54321)

# Luego, dividimos el conjunto de validación + prueba en validación y prueba
features_val, features_test, target_val, target_test = train_test_split(
    features_valtest, target_valtest, test_size=0.50, random_state=54321)

## 4. Prueba de modelos

### 4.1 Árbol de decisión

In [5]:
# Probamos el modelo de Árbol de Decisión con dephts de 1 a 5
best_model_dt = None
best_accuracy_dt = 0
best_depth_dt = 0

for depth in range(1, 6):
    model_dt = DecisionTreeClassifier(max_depth=depth, random_state=54321)
    model_dt.fit(features_train, target_train)
    accuracy = model_dt.score(features_val, target_val)

    if accuracy > best_accuracy_dt:
        best_model_dt = model_dt
        best_accuracy_dt = accuracy
        best_depth_dt = depth

print(
    f"Exactitud del mejor modelo en el conjunto de validación para Árbol de decisión (max_depth = {best_depth_dt}): {best_accuracy_dt:.4f}")

Exactitud del mejor modelo en el conjunto de validación para Árbol de decisión (max_depth = 3): 0.7652


### 4.2 Random Forest

In [6]:
# Probamos el modelo de Bosque Aleatorio con dephts de 1 a 5 y n_estimators de 10 a 50
best_model_rf = None
best_accuracy_rf = 0
best_depth_rf = 0
best_n_estimators_rf = 0

for depth in range(1, 6):
    for n_estimators in range(10, 51, 10):
        model_rf = RandomForestClassifier(
            max_depth=depth, n_estimators=n_estimators, random_state=54321)
        model_rf.fit(features_train, target_train)
        accuracy_rf = model_rf.score(features_val, target_val)

        if accuracy_rf > best_accuracy_rf:
            best_model_rf = model_rf
            best_accuracy_rf = accuracy_rf
            best_depth_rf = depth
            best_n_estimators_rf = n_estimators
print(
    f"Exactitud del mejor modelo en el conjunto de validación para Bosque Aleatorio (max_depth = {best_depth_rf}, n_estimators = {best_n_estimators_rf}): {best_accuracy_rf:.4f}")

Exactitud del mejor modelo en el conjunto de validación para Bosque Aleatorio (max_depth = 4, n_estimators = 10): 0.7854


### 4.3 Regresión logística

In [7]:
# Probamos el modelo de Regresión Logística
model_lr = LogisticRegression(solver='liblinear', random_state=54321)
model_lr.fit(features_train, target_train)
accuracy_lr = model_lr.score(features_val, target_val)

print(
    f"Exactitud del modelo de Regresión Logística en el conjunto de validación: {accuracy_lr:.4f}")

Exactitud del modelo de Regresión Logística en el conjunto de validación: 0.6781


### Conclusiones

- Como podemos notar al probar la calidad de diferentes modelos cambiando los hiperparámetros, el mejor modelo para este dataset fué el de Bosque Aleatorio con max_depth=4 y n_estimators=10.
- El modelo de Árbol de decisión tuvo un accuracy bastante alto y cercano al Bosque Aleatorio, que podría tomarse en cuenta si se busca más rapidez del modelo.
- El modelo de Regresión Logística presentó la menor accuracy.

## 5. Prueba de cordura

In [8]:
# Prueba de cordura
#  Encontrar la clase mayoritaria en entrenamiento
predictions_cord = np.full_like(
    target_test, fill_value=Counter(target_train).most_common(1)[0][0])
accuracy_cord = accuracy_score(target_test, predictions_cord)

print(f"Exactitud del modelo de cordura (mayoría): {accuracy_cord:.4f}")

Exactitud del modelo de cordura (mayoría): 0.7247


## 6. Mejor modelo

In [9]:
# Mostrar el mejor modelo
best_model = None
if best_accuracy_dt >= best_accuracy_rf and best_accuracy_dt >= accuracy_lr:
    best_model = best_model_dt
    print(
        f"El mejor modelo es el Árbol de Decisión con una precisión de {best_accuracy_dt:.4f} en el conjunto de validación.")
elif best_accuracy_rf >= best_accuracy_dt and best_accuracy_rf >= accuracy_lr:
    best_model = best_model_rf
    print(
        f"El mejor modelo es el Bosque Aleatorio con una precisión de {best_accuracy_rf:.4f} en el conjunto de validación.")
else:
    best_model = model_lr
    print(
        f"El mejor modelo es el de Regresión Logística con una precisión de {accuracy_lr:.4f} en el conjunto de validación.")

El mejor modelo es el Bosque Aleatorio con una precisión de 0.7854 en el conjunto de validación.


- Como podemos notar al probar la calidad de diferentes modelos cambiando los hiperparámetros, el mejor modelo para este dataset fué el de Bosque Aleatorio con max_depth=4 y n_estimators=10.
- El modelo de Árbol de decisión tuvo un accuracy bastante alto y cercano al Bosque Aleatorio, que podría tomarse en cuenta si se busca más rapidez del modelo.
- El modelo de Regresión Logística presentó la menor accuracy.

In [10]:
# Comprobar la calidad del modelo con el conjunto de prueba
test_accuracy = best_model.score(features_test, target_test)
print(
    f"Precisión del mejor modelo en el conjunto de prueba: {test_accuracy:.4f}")

Precisión del mejor modelo en el conjunto de prueba: 0.8180
